In [1]:
# Импорт библиотек
from numba import njit, prange
from typing import Literal
from typing_extensions import Self
from numpy import empty, argsort, append, array
from numpy.typing import NDArray
import numpy as np
from random import choice
import os
import time

In [22]:
# encode_endings, read_endings
def encode_endings(endings):
	'''
	Функция кодировки данных из дата-сета числами
	'b' = 0
	'x' = 1
	'o' = 2
	'neutral' = 3
	'positive' = 4
	'negative' = 5
	'''
	encoded_type = '10u1'
	endings_count = len(endings)
	result = empty(endings_count, dtype=encoded_type)
	for i in range(endings_count):
		ending = endings[i]
		temp = []
		for figure in ending[:9]:
			if figure == 'b':
				temp.append(0)
			elif figure == 'x':
				temp.append(1)
			elif figure == 'o':
				temp.append(2)

		if ending[9] == 'neutral':
			temp.append(3)
		elif ending[9] == 'positive':
			temp.append(4)
		elif ending[9] == 'negative':
			temp.append(5)
		result[i] = temp
	return result

def read_endings():
	'''
	Функция чтения дата-сета
	'''
	source = np.genfromtxt('tic-tac-toe.data', delimiter=',', dtype='U30')
	for ending in source:
		if ending[9] == 'negative' and 'b' not in ending:
			ending[9] = 'neutral'
	return encode_endings(source)

endings = read_endings()

In [23]:
# get_situation
@njit()
def get_situation(x_moves, o_moves):
	'''
	Функция преобразующая историю ходов в текущую позицию на карте
	'''
	return np.array([1 if (i in x_moves) else 2 if (i in o_moves) else 0 for i in range(9)])

In [24]:
# get_ending_distance
@njit()
def get_ending_distance(situation, ending, player_is_x):
	'''
	Функция вычисляющая число ходов необходимых игроку для достижения концовки из текущей позиции.
	Если концовка не достижима возвращает -1
	'''
	ending_distance = 0
	for i in range(9):
		if situation[i] != ending[i]:
			if situation[i] == 0:
				if player_is_x and ending[i] == 1:
					ending_distance += 1
				if not player_is_x and ending[i] == 2:
					ending_distance += 1
			else:
				return -1
	return ending_distance

In [25]:
# get_situation_endings
@njit()
def get_situation_endings(endings, situation, player_is_x):
	'''
	Отсеивает недостижимые концовки. 
	Адаптирует результат под то "кем" играет игрок.
	Сортирует концовки по расстоянию до них.
	'''
	endings_result_count = 0
	endings_count = len(endings)
	result_endings = np.empty((endings_count, 11), dtype='u1')
	for i in range(endings_count):
		ending = endings[i]
		result = ending[9]
		ending_distance = get_ending_distance(situation, ending, player_is_x)
		if ending_distance <= 0:
			continue
		# Инвертируем результат если играет "нолик"
		if player_is_x == False:
			if result == 5:
				result = 3
			elif result == 3:
				result = 5
		result_endings[endings_result_count, :] = [ending_distance, result, *ending[:9]]
		endings_result_count += 1
		
	return result_endings[argsort(result_endings[:endings_result_count,0], kind='mergesort')]


In [26]:
# get_move_endings
@njit()
def get_move_endings(result_endings, move, situation, player_is_x):
	'''
	Отсеивает недостижимые концовки. 
	Из информации по каждой концовке оставляет только расстояние и результат. 
	'''
	move_situation = [situation[i] if i != move else 1 if player_is_x else 2 for i in range(9)]
	endings_result_count = len(result_endings)
	move_endings_count = 0
	move_endings = empty((endings_result_count, 2), dtype='u1')
	for e_i in range(endings_result_count):
		ending = result_endings[e_i]
		ending_situation = ending[2:11]
		dist = get_ending_distance(move_situation, ending_situation, player_is_x)
		if dist == -1:
			continue
		result = ending[1]
		move_endings[move_endings_count, :] = [dist, result]
		move_endings_count += 1
	return move_endings[:move_endings_count]

In [27]:
# analyse
@njit()
def analyse(endings, player_is_x, x_moves, o_moves, allowed_moves):
	'''
	Производит анализ доступных ходов вычисляя для них следующие метрики:

	0. move
	1. nearest_draw
	2. nearest_win
	3. nearest_loose
	4. max_draw_rate
	5. max_win_rate
	6. max_loose_rate
	7. avg_draw_rate
	8. avg_win_rate
	9. avg_loose_rate
	10. min_draw_rate
	11. min_win_rate
	12. min_loose_rate
	13. total_draw_rate
	14. total_win_rate
	15. total_loose_rate
	'''
	situation = get_situation(x_moves, o_moves)
	result_endings = get_situation_endings(endings, situation, player_is_x)
	moves_count = len(allowed_moves)
	analyzed_moves = empty((moves_count, 13), dtype='f4')
	# Перебираем доступные ходы и доступные для них концовки
	for m_i in range(moves_count):
		move = allowed_moves[m_i]
		# Отфильтровываем недостижимые концовки
		# [dist, result]
		move_endings = get_move_endings(result_endings, move, situation, player_is_x)
		move_endings_count = len(move_endings)
		# Возможна ситуация когда концовка уже достигнута, тогда возвращаем пустой массив
		if move_endings_count == 0:
			return empty((0, 16), dtype='f4')
		# Вычисляем число групп концовок по расстоянию
		min_dist = min(move_endings[:, 0])
		dist_count = max(move_endings[:, 0]) - min_dist + 1
		dist_metrics = np.zeros((dist_count, 7), dtype='f4')
		# Инициализируем счётчики расстояния максимально возможным значением
		nearest_draw = 4
		nearest_win = 4
		nearest_loose = 4
		# Группируем концовки по расстоянию до них, вычисляя статистику
		for e_i in range(move_endings_count):
			dist, result = move_endings[e_i]
			d_i = dist - min_dist
			# Ничья - 3
			if result == 3 and nearest_draw > dist:
				nearest_draw = dist
			# Победа - 4
			elif result == 4 and nearest_win > dist:
				nearest_win = dist
			# Поражение - 5
			elif result == 5 and nearest_loose > dist:
				nearest_loose = dist
			
			# Вычисляем статистику
			# 3 - 3 = 0; draw_count
			# 4 - 3 = 1; win_count
			# 5 - 3 = 2; loose_count
			#		  3; total_count
			#		  4; draw_rate
			#		  5; win_rate
			#		  6; lose_rate
			metric_i = result - 3
			dist_metrics[d_i, metric_i] += 1
			dist_metrics[d_i, 3] += 1

		# Считаем проценты
		#		  3; total_count
		#		  4; draw_rate
		#		  5; win_rate
		#		  6; lose_rate
		for d_i in range(dist_count):
			for metric_i in range(3):
				dist_metrics[d_i, 4 + metric_i] = dist_metrics[d_i, metric_i] / dist_metrics[d_i, 3] * 100
		
		# Считаем метрики
		# move			  0
		#
		# nearest_draw	  1
		# nearest_win	  2
		# nearest_loose	  3
		#
		# 3 * 0 + 0 + 4 = 4; max_draw_rate
		# 3 * 0 + 1 + 4 = 5; max_win_rate
		# 3 * 0 + 2 + 4 = 6; max_loose_rate
		#
		# 3 * 1 + 0 + 4 = 7; avg_draw_rate
		# 3 * 1 + 1 + 4 = 8; avg_win_rate
		# 3 * 1 + 2 + 4 = 9; avg_loose_rate
		#
		# 3 * 2 + 0 + 4 = 10; min_draw_rate
		# 3 * 2 + 1 + 4 = 11; min_win_rate
		# 3 * 2 + 2 + 4 = 12; min_loose_rate
		#
		# 3 * 3 + 0 + 4 = 13; total_draw_rate
		# 3 * 3 + 1 + 4 = 14; total_win_rate
		# 3 * 3 + 2 + 4 = 15; total_loose_rate
		analyzed_moves[m_i, 0:4] = [move, nearest_draw, nearest_loose, nearest_win]
		for i in range(3):
			percents = dist_metrics[:, i + 4]
			absolute = dist_metrics[:, i]
			total = dist_metrics[:, 3]
			analyzed_moves[m_i, 3 * 0 + i + 4] = max(percents)
			analyzed_moves[m_i, 3 * 1 + i + 4] = sum(percents) / len(percents)
			analyzed_moves[m_i, 3 * 2 + i + 4] = min(percents)
			analyzed_moves[m_i, 3 * 3 + i + 4] = sum(absolute) / sum(total) * 100
	return analyzed_moves


In [28]:
# analyse_and_use_metrics
@njit
def analyse_and_use_metrics(metrics, endings, player_is_x, x_moves, o_moves, allowed_moves):
	analyse_result = analyse(endings, player_is_x, x_moves, o_moves, allowed_moves)
	if len(analyse_result) <= 1:
		return analyse_result
	count = len(metrics)
	for i in prange(count):
		metric = metrics[i] # 1 -> asc; -1 -> desc
		n = abs(metric)	 	# индекс метрики
		column = analyse_result[:, n]
		value = min(column) if metric < 0 else max(column)
		indices = np.where(column == value)
		analyse_result = analyse_result[indices]
		if len(analyse_result) == 1:
			return analyse_result
	return analyse_result


In [29]:
# Types
PlayerId = str
PlayerMove = Literal[0, 1, 2, 3, 4, 5, 6, 7, 8]
MovesList = NDArray[np.int8]

In [30]:
class Player:
	_name: PlayerId
	_draw_counter: int
	_win_counter: int
	_loose_counter: int
	_moves: MovesList
	_original: Self
	_opponent_name: str
	_copies: list[tuple[Self, Self]]
	_is_x_player: bool
	_is_original: bool

	def __init__(self, name: PlayerId):
		self._name = name
		self._draw_counter = None
		self._win_counter = None
		self._loose_counter = None
		self._total_counter = None
		self._moves = None
		self._original = None
		self._opponent_name = None
		self._copies = None
		self._is_x_player = None
		self._is_original = True

	def __getitem__(self, index) -> None|Self:
		if self._copies is None:
			return None
		return self._copies[index]
	
	def __setitem__(self, index, value):
		if self._copies is None:
			return None
		self._copies[index] = value
	
	def __make_copy(self, opponent: Self, is_x_player: bool):
		copy = type(self)(self._name)
		copy._opponent_name = opponent._name
		copy._is_x_player = is_x_player
		copy._copy_handler(self)
		return copy
	
	def _copy_handler(self, original: Self):
		self._original = original
		self._draw_counter = 0
		self._win_counter = 0
		self._loose_counter = 0
		self._total_counter = 0
		self._is_original = False
	
	def prepare_for_games(self, opponents: list[Self]):
		opponents_count = len(opponents)
		self._copies = list([(None, None)] * opponents_count)
		for i in prange(opponents_count):
			opponent = opponents[i]
			self[i] = (
				self.__make_copy(opponent, True), 
				self.__make_copy(opponent, False),
			)

	def clear_score(self, id_range):
		os.makedirs(f'scores_{id_range}', exist_ok=True)
		with open(f'scores_{id_range}/{self._name}.log', 'w') as file:
			# title = ','.join(map(
			# 	lambda copies: f'X vs {copies[0]._opponent_name},O vs {copies[1]._opponent_name}', 
			# 	self._copies
			# ))
			# file.write(f'{title},')
			file.write('total\n')
			
	def save_score(self, id_range):
		score_lines = []
		draw_counter = 0
		win_counter = 0
		loose_counter = 0
		total_counter = 0
		for copies in self._copies:
			draw_counter += copies[0]._draw_counter + copies[1]._draw_counter
			win_counter += copies[0]._win_counter + copies[1]._win_counter
			loose_counter += copies[0]._loose_counter + copies[1]._loose_counter
			total_counter += copies[0]._total_counter + copies[1]._total_counter
			# score_lines.append(f'{copies[0]._draw_counter}|{copies[0]._win_counter}|{copies[0]._loose_counter}|{copies[0]._total_counter}')
			# score_lines.append(f'{copies[1]._draw_counter}|{copies[1]._win_counter}|{copies[1]._loose_counter}|{copies[1]._total_counter}')

		with open(f'scores_{id_range}/{self._name}.log', 'a') as file:
			# file.write(','.join(score_lines))
			file.write(f'{draw_counter}|{win_counter}|{loose_counter}|{total_counter}')
			file.write('\n')

	def new_game_handler(self):
		self._moves = empty(0, dtype='u1')

	def make_move(self, opponent_moves: MovesList, allowed_moves: MovesList) -> PlayerMove:
		assert type(self) != Player, 'Player не может ходить!'
		assert self._is_original == False, 'Оригинал не может ходить!'

	def move_handler(self, move: PlayerMove):
		self._moves = append(self._moves, array([move], dtype='u1'))

	def draw_handler(self):
		assert type(self) != Player, 'Player не может выйти в ничью!'
		assert self._is_original == False, 'Оригинал не может выйти в ничью!'
		self._draw_counter += 1
		self._total_counter += 1

	def win_handler(self):
		assert type(self) != Player, 'Player не может победить!'
		assert self._is_original == False, 'Оригинал не может победить!'
		self._win_counter += 1
		self._total_counter += 1

	def loose_handler(self):
		assert type(self) != Player, 'Player не может проиграть!'
		assert self._is_original == False, 'Оригинал не может проиграть!'
		self._loose_counter += 1
		self._total_counter += 1

	def __str__(self):
		result = f'{self.__class__.__name__} {self._name}'
		if self._is_original:
			result += ' Origin'
		else:
			result += f'[{"X" if self._is_x_player else "O"}]'
			result += f' vs {self._opponent_name}'
			result += f'[{"O" if self._is_x_player else "X"}]'
		return f'{{{result}}}' 


In [31]:
class RandomPlayer(Player):
	def make_move(self, opponent_moves: MovesList, allowed_moves: MovesList) -> PlayerMove:
		super().make_move(opponent_moves, allowed_moves)
		return choice(allowed_moves)

In [32]:
class AnalyticPlayer(Player):
	_metrics: list[int]
	def __init__(self, name: str, metrics: list[int]=None):
		super().__init__(name)
		self._metrics = metrics
		if metrics is not None:
			names = [
				'move',
				'n_d',
				'n_w',
				'n_l',
				'ma_d_r',
				'ma_w_r',
				'ma_l_r',
				'a_d_r',
				'a_w_r',
				'a_l_r',
				'mi_d_r',
				'mi_w_r',
				'mi_l_r',
				't_d_r',
				't_w_r',
				't_l_r',
			]
			self._name += ' ' + ' '.join(map(
				lambda m: f'{"+" if m > 0 else "-"}{names[abs(m)]}', 
				metrics
			))

	def _copy_handler(self, original: Self):
		super()._copy_handler(original)
		self._metrics = [m for m in original._metrics]

	def make_move(self, opponent_moves: MovesList, allowed_moves: MovesList) -> PlayerMove:
		super().make_move(opponent_moves, allowed_moves)
		x_moves = None
		o_moves = None
		if self._is_x_player:
			x_moves = self._moves
			o_moves = opponent_moves
		else:
			o_moves = self._moves
			x_moves = opponent_moves
		
		analyse_result = analyse_and_use_metrics(
			self._metrics, endings, self._is_x_player, 
			x_moves, o_moves, allowed_moves
		)
		assert len(analyse_result) != 0, 'Игра уже окончена!'
		# if len(analyse_result) > 1:
		# 	print(f'{self}: Выбираю 1 из {len(analyse_result)}')
		return int(choice(analyse_result)[0])

In [33]:
# check_win
@njit
def check_win(game_board, figure):
	# 3 по 1 горизонтали
	if (game_board[0] == figure and
		game_board[1] == figure and
		game_board[2] == figure):
		return True
	# 3 по 2 горизонтали
	if (game_board[3] == figure and
		game_board[4] == figure and
		game_board[5] == figure):
		return True
	# 3 по 3 горизонтали
	if (game_board[6] == figure and
		game_board[7] == figure and
		game_board[8] == figure):
		return True
	
	# 3 по 1 вертикали
	if (game_board[0] == figure and
		game_board[3] == figure and
		game_board[6] == figure):
		return True
	# 3 по 2 вертикали
	if (game_board[1] == figure and
		game_board[4] == figure and
		game_board[7] == figure):
		return True
	# 3 по 3 вертикали
	if (game_board[2] == figure and
		game_board[5] == figure and
		game_board[8] == figure):
		return True
	# 3 по 1 вертикали
	if (game_board[0] == figure and
		game_board[3] == figure and
		game_board[6] == figure):
		return True
	
	# 3 по 1 диагонали
	if (game_board[0] == figure and
		game_board[4] == figure and
		game_board[8] == figure):
		return True
	# 3 по 2 диагонали
	if (game_board[2] == figure and
		game_board[4] == figure and
		game_board[6] == figure):
		return True
	
	return False

In [14]:
# play_game
def play_game(x_player: Player, o_player: Player):
	game_board = np.array([
		'b', 'b', 'b',
		'b', 'b', 'b',
		'b', 'b', 'b',
	])
	allowed_moves = np.array([
		0, 1, 2, 
		3, 4, 5, 
		6, 7, 8
	])
	x_player.new_game_handler()
	o_player.new_game_handler()
	player = x_player
	other_player = o_player
	while True:
		figure = 'x' if player._is_x_player else 'o'
		move = player.make_move(other_player._moves, allowed_moves)
		if move not in allowed_moves:
			raise IndexError(f'Invalid move! {move}, allowed: {allowed_moves}')
		player.move_handler(move)
		game_board[move] = figure
		allowed_moves = allowed_moves[np.where(allowed_moves != move)]
		if check_win(game_board, figure):
			player.win_handler()
			other_player.loose_handler()
			return
		if len(allowed_moves) == 0:
			player.draw_handler()
			other_player.draw_handler()
			return
		player, other_player = other_player, player


In [15]:
# a = AnalyticPlayer('', [-6, 14, 13])
# a.prepare_for_games([a])
# a.clear_score()
# a.save_score()
# for i in prange(100):
# 	play_game(a[0][0], a[0][1])
# 	a.save_score()

In [16]:
# print(a[0][0]._win_counter)
# print(a[0][0]._draw_counter)
# print(a[0][0]._loose_counter)

In [17]:
while True:
	pass

KeyboardInterrupt: 

In [9]:
players: list[Player] = [
	RandomPlayer('0001 random'),
]

_index = 2
for metrics in np.array(np.meshgrid(
		[-1, 1, 2, -3, -4, 4, 5, -6, -7, 7, 8, -9, -10, 10, 11, -12, -13, 13, 14, -15],
	)).T.reshape(-1, 1):
	players.append(AnalyticPlayer(str(_index).zfill(4), metrics))
	_index += 1

for metrics in np.array(np.meshgrid(
		[-1, 1, 2, -3, -4, 4, 5, -6, -7, 7, 8, -9, -10, 10, 11, -12, -13, 13, 14, -15],
		[-1, 1, 2, -3, -4, 4, 5, -6, -7, 7, 8, -9, -10, 10, 11, -12, -13, 13, 14, -15],
	)).T.reshape(-1, 2):
	if len(np.unique(metrics)) != 2:
		continue
	players.append(AnalyticPlayer(str(_index).zfill(4), metrics))
	_index += 1

for metrics in np.array(np.meshgrid(
		[-1, 1, 2, -3, -4, 4, 5, -6, -7, 7, 8, -9, -10, 10, 11, -12, -13, 13, 14, -15],
		[-1, 1, 2, -3, -4, 4, 5, -6, -7, 7, 8, -9, -10, 10, 11, -12, -13, 13, 14, -15],
		[-1, 1, 2, -3, -4, 4, 5, -6, -7, 7, 8, -9, -10, 10, 11, -12, -13, 13, 14, -15],
	)).T.reshape(-1, 3):
	if len(np.unique(metrics)) != 3:
		continue
	players.append(AnalyticPlayer(str(_index).zfill(4), metrics))
	_index += 1

In [ ]:
_index = 0
for player in players[:1000]:
	player.prepare_for_games(players)
	_index += 1
	print(f'\r {_index}   ')

In [ ]:
_index = 0
start_time = time.time()
for i in prange(1000):
	play_game(players[i][i][0], players[i][i][1])
	players[i].save_score()
	for ii in prange(i+1, 1000):
		play_game(players[i][ii][0], players[ii][i][1])
		players[i].save_score()
		players[ii].save_score()
		play_game(players[ii][i][0], players[i][ii][1])
		players[i].save_score()
		players[ii].save_score()
	_index += 1
	seconds = time.time() - start_time
	minutes = int(seconds // 60)
	seconds -= minutes * 60
	print(f'{_index} {minutes}:{int(seconds)}')

In [ ]:
selection = players[:1000]
for player in selection:
	player._draw_counter = 0
	player._win_counter = 0
	player._loose_counter = 0
	player._total_counter = 0
	for copy in player._copies:
		player._draw_counter += copy[0]._draw_counter + copy[1]._draw_counter
		player._win_counter += copy[0]._win_counter + copy[1]._win_counter
		player._loose_counter += copy[0]._loose_counter + copy[1]._loose_counter
		player._total_counter += copy[0]._total_counter + copy[1]._total_counter

selection.sort(key=lambda p: (-p._win_counter, -p._draw_counter))
print(*map(lambda p: f'{p} w: {p._win_counter} d: {p._draw_counter} l: {p._loose_counter}', selection), sep='\n')

In [41]:
for player in players:
	if player is None:
		continue
	if hasattr(player, '_copies') and player._copies is not None:
		for copy in player._copies:
			del copy
		del player._copies
	del player
del players


In [11]:
players: list[Player] = [
	RandomPlayer('0001 random'),
]

_index = 2
print(len(players))
for metrics in np.array(np.meshgrid(
		[-1, 1, 2, -3, -4, 4, 5, -6, -7, 7, 8, -9, -10, 10, 11, -12, -13, 13, 14, -15],
	)).T.reshape(-1, 1):
	players.append(AnalyticPlayer(str(_index).zfill(4), metrics))
	_index += 1
print(len(players))
for metrics in np.array(np.meshgrid(
		[-1, 1, 2, -3, -4, 4, 5, -6, -7, 7, 8, -9, -10, 10, 11, -12, -13, 13, 14, -15],
		[-1, 1, 2, -3, -4, 4, 5, -6, -7, 7, 8, -9, -10, 10, 11, -12, -13, 13, 14, -15],
	)).T.reshape(-1, 2):
	if len(np.unique(metrics)) != 2:
		continue
	players.append(AnalyticPlayer(str(_index).zfill(4), metrics))
	_index += 1
print(len(players))
for metrics in np.array(np.meshgrid(
		[-1, 1, 2, -3, -4, 4, 5, -6, -7, 7, 8, -9, -10, 10, 11, -12, -13, 13, 14, -15],
		[-1, 1, 2, -3, -4, 4, 5, -6, -7, 7, 8, -9, -10, 10, 11, -12, -13, 13, 14, -15],
		[-1, 1, 2, -3, -4, 4, 5, -6, -7, 7, 8, -9, -10, 10, 11, -12, -13, 13, 14, -15],
	)).T.reshape(-1, 3):
	if len(np.unique(metrics)) != 3:
		continue
	players.append(AnalyticPlayer(str(_index).zfill(4), metrics))
	_index += 1
print(len(players))
print(_index)

1
21
401
7241
7242


In [35]:
from test_model import test_model
class AnalyticModel:
	def __init__(self, name, metrics):
		self.player = AnalyticPlayer(name, metrics)
		self.player._is_original = False
	def make_move(self, x_moves, o_moves, model_is_x, allowed_moves):
		x_moves_array = np.array(x_moves)
		o_moves_array = np.array(o_moves)
		allowed_moves_array = np.array(allowed_moves)
		analyse_result = analyse_and_use_metrics(
			self.player._metrics, endings, model_is_x, 
			x_moves_array, o_moves_array, allowed_moves_array
		)
		assert len(analyse_result) != 0, 'Игра уже окончена!'
		# if len(analyse_result) > 1:
		# 	print(f'{self}: Выбираю 1 из {len(analyse_result)}')
		return int(choice(analyse_result)[0])

model = AnalyticModel('AnalyticModel_1', [+11, -9, -12])
stats_x, logs_x = test_model(model, True, 12)
print(f"As X: total={stats_x['total']}, wins={stats_x['wins']}, draws={stats_x['draws']}, losses={stats_x['losses']}")
stats_o, logs_o = test_model(model, False, 12)
print(f"As O: total={stats_o['total']}, wins={stats_o['wins']}, draws={stats_o['draws']}, losses={stats_o['losses']}")

with open('model_endings.log', 'a') as f:
	f.write(f'# {model.player._name}\n')
	f.write('# Model played as X\n')
	for x_moves, o_moves, is_x, status in logs_x:
		f.write(f"X_moves={x_moves}, O_moves={o_moves}, model_is_x={is_x}, status={status}\n")
	f.write('# Model played as O\n')
	for x_moves, o_moves, is_x, status in logs_o:
		f.write(f"X_moves={x_moves}, O_moves={o_moves}, model_is_x={is_x}, status={status}\n")
print('Testing completed. Logs written to model_endings.log')

As X: total=93, wins=89, draws=4, losses=0
As O: total=885, wins=0, draws=13, losses=872
Testing completed. Logs written to model_endings.log


In [19]:
start = 2998
end = 3997
players = [players[0]] + players[start:]
print(len(players))
for i in prange(5):
	selection = players[:1000]
	if len(selection) == 1:
		break
	_index = 0
	id_range = f'{start+1}-{end}'
	os.system(f'rd /s /q scores_{id_range}')
	print(f'preparing {id_range}')
	for player in selection:
		player.prepare_for_games(selection)
		player.clear_score(id_range)
		_index += 1
		print(f'{_index}')

	_index = 0
	start_time = time.time()
	print(f'testing {id_range}')
	count = len(selection)
	for i in prange(count):
		play_game(selection[i][i][0], selection[i][i][1])
		selection[i].save_score(id_range)
		for ii in prange(i+1, count):
			play_game(selection[i][ii][0], selection[ii][i][1])
			selection[i].save_score(id_range)
			selection[ii].save_score(id_range)
			play_game(selection[ii][i][0], selection[i][ii][1])
			selection[i].save_score(id_range)
			selection[ii].save_score(id_range)
		_index += 1
		seconds = time.time() - start_time
		minutes = int(seconds // 60)
		seconds -= minutes * 60
		print(f'{_index} {minutes}:{int(seconds)}')
	
	for player in selection:
		player._draw_counter = 0
		player._win_counter = 0
		player._loose_counter = 0
		player._total_counter = 0
		for copy in player._copies:
			player._draw_counter += copy[0]._draw_counter + copy[1]._draw_counter
			player._win_counter += copy[0]._win_counter + copy[1]._win_counter
			player._loose_counter += copy[0]._loose_counter + copy[1]._loose_counter
			player._total_counter += copy[0]._total_counter + copy[1]._total_counter

	selection.sort(key=lambda p: (-p._win_counter, -p._draw_counter))
	with open(f'score_board_{id_range}.log', 'w') as file:
		file.writelines(map(lambda p: f'{p} w: {p._win_counter} d: {p._draw_counter} l: {p._loose_counter}\n', selection))
	
	for player in players[1:1000]:
		for copy in player._copies:
			del copy
		del player._copies
		del player
	del players[1:1000]
	start = end
	end = end + 999

4244
preparing 2999-3997
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271

In [30]:
players: list[Player] = [
	RandomPlayer('0001 random'),
]

# 0. move
# 1. nearest_draw
# 2. nearest_win
# 3. nearest_loose
# 4. max_draw_rate
# 5. max_win_rate
# 6. max_loose_rate
# 7. avg_draw_rate
# 8. avg_win_rate
# 9. avg_loose_rate
# 10. min_draw_rate
# 11. min_win_rate
# 12. min_loose_rate
# 13. total_draw_rate
# 14. total_win_rate
# 15. total_loose_rate

# Первый
# Результаты отбора
# ('+mi_d_r', 0.6255785510604176, 0.33739529495633575) 	10
# ('+a_d_r', 0.5832034842274104, 0.32250178221350917)	7
# ('-n_d', 0.5701270718232048, 0.3903729281767954)		-1
# ('-a_l_r', 0.5630933434325434, 0.4313256549634648)	-9
# ('-ma_l_r', 0.5625587684904646, 0.43159365531990757)	-6
# ('+n_w', 0.5364267955801104, 0.42882044198895003)		2
# ('-t_d_r', 0.5069081268936019, 0.4634743361254679)	-13
# ('+a_w_r', 0.49988121546961306, 0.4999419443949385)	8
# ('+mi_w_r', 0.49968646408839784, 0.4999460434860097)	11
# ('+ma_d_r', 0.4958943147389057, 0.4207879165924081)	4

# Результаты финала
# ('+mi_d_r', 0.5717747949291574, 0.41181953765846385)
# ('+n_w', 0.5536188972233188, 0.43352414791419924)
# ('-ma_l_r', 0.542906961442295, 0.4568430056586393)
# ('-a_l_r', 0.5423345176996973, 0.4574220292143703)
# ('-n_d', 0.528090495778307, 0.44977869187654856)
# ('+mi_w_r', 0.49998684037373337, 0.4999013028030004)
# ('+a_w_r', 0.49992762205553365, 0.4999473614949335)
# ('+a_d_r', 0.45941882562363184, 0.5138618267542278)
# ('+ma_d_r', 0.45567837873404393, 0.5278326095538887)
# ('-t_d_r', 0.3071276458440888, 0.6471562553777319)
_index = 2
for metrics in np.array(np.meshgrid(
		[10, 7, -1, -9, -6, 2, -13, 8, 11, 4],
	)).T.reshape(-1, 1):
	players.append(AnalyticPlayer(str(_index).zfill(4), metrics))
	_index += 1

# Второй
# Результаты отбора
# ('-a_l_r', 0.5251163881690644, 0.4527180323474222)	-9
# ('-ma_l_r', 0.5247923331248324, 0.45329961576266664)	-6
# ('+n_w', 0.5094661781788934, 0.4567660620141184)		2
# ('-n_l', 0.5020083996068272, 0.4714979000982933)		-3
# ('+mi_w_r', 0.497662183897775, 0.48495697435439217)	11
# ('+a_w_r', 0.49468094897685655, 0.4883033687784823)	8
# ('+a_d_r', 0.4945493700294882, 0.4469721204539357)	7
# ('-n_d', 0.4935831918505942, 0.46649486194263257)		-1
# ('+mi_d_r', 0.49155464212313454, 0.4684818604235548)	10
# ('+ma_d_r', 0.48917402376909985, 0.45237087838441614)	4

# Результаты финала
# ('-ma_l_r', 0.5071193578102382, 0.4803131991051454)
# ('-a_l_r', 0.5068101065929728, 0.4806092906961442)
# ('+n_w', 0.49872351625213845, 0.4859126200815897)
# ('-n_d', 0.49627745303216186, 0.4869199682470953)
# ('+ma_d_r', 0.49205816554809845, 0.49149230161863405)
# ('+a_d_r', 0.49132207548531426, 0.49105145413870244)
# ('-n_l', 0.4900697320196106, 0.49321719263172925)
# ('+a_w_r', 0.4846821950256613, 0.5050401368601132)
# ('+mi_d_r', 0.4835341945105963, 0.4999458757306776)
# ('+mi_w_r', 0.4827543097776023, 0.5047045663903145)
for metrics in np.array(np.meshgrid(
		[10, 7, -1, -9, -6, 2, -13, 8, 11, 4],
		[-9, -6, 2, -3, 11, 8, 7, -1, 10, 4],
	)).T.reshape(-1, 2):
	if len(np.unique(metrics)) != 2:
		continue
	players.append(AnalyticPlayer(str(_index).zfill(4), metrics))
	_index += 1

# Третий
# Результаты отбора
# ('-ma_l_r', 0.5017266081871347, 0.4698084795321638)	-6
# ('-a_l_r', 0.4988947368421052, 0.47376900584795295)	-9
# ('+mi_w_r', 0.49825730994152023, 0.47686988304093547)	11
# ('-n_l', 0.49157456140350897, 0.4768479532163741)		-3
# ('+a_w_r', 0.49105847953216375, 0.4815730994152047)	8
# ('+n_w', 0.489387426900585, 0.47665058479532124)		2
# ('+t_w_r', 0.48781871345029265, 0.47487134502923994)	14
# ('+ma_w_r', 0.48281871345029215, 0.48882017543859657)	5
# ('-mi_l_r', 0.48203947368421024, 0.48973099415204646)	-12
# ('+ma_d_r', 0.4813757309941523, 0.4762807017543863)	4

# Результаты финала
# ('+ma_d_r', 0.5043133829793754, 0.47841776225062055)
# ('-a_l_r', 0.4992108730961356, 0.48743526094817813)
# ('-ma_l_r', 0.49885078606233335, 0.4876421194569581)
# ('-n_l', 0.49851548043196064, 0.48588670699036096)
# ('+a_w_r', 0.49598541264441787, 0.49110508412246023)
# ('+mi_w_r', 0.49499708865802455, 0.49141920259875577)
# ('+n_w', 0.49131194263124023, 0.4926220465201802)
# ('+ma_w_r', 0.49129729331071614, 0.4954949971728495)
# ('-mi_l_r', 0.487031983676279, 0.4980148486859896)
# ('+t_w_r', 0.4834550237234801, 0.5024952675959388)
for metrics in np.array(np.meshgrid(
		[10, 7, -1, -9, -6, 2, -13, 8, 11, 4],
		[-9, -6, 2, -3, 11, 8, 7, -1, 10, 4],
		[-6, -9, 11, -3, 8, 2, 14, 5, -12, 4],
	)).T.reshape(-1, 3):
	if len(np.unique(metrics)) != 3:
		continue
	players.append(AnalyticPlayer(str(_index).zfill(4), metrics))
	_index += 1
print(_index)

895


In [31]:
selection = players
_index = 0
id_range = f'FINAL'
os.system(f'rd /s /q scores_{id_range}')
print(f'preparing {id_range}')
for player in selection:
	player.prepare_for_games(selection)
	player.clear_score(id_range)
	_index += 1
	print(f'{_index}')

_index = 0
start_time = time.time()
print(f'testing {id_range}')
count = len(selection)
for i in prange(count):
	play_game(selection[i][i][0], selection[i][i][1])
	selection[i].save_score(id_range)
	for ii in prange(i+1, count):
		play_game(selection[i][ii][0], selection[ii][i][1])
		selection[i].save_score(id_range)
		selection[ii].save_score(id_range)
		play_game(selection[ii][i][0], selection[i][ii][1])
		selection[i].save_score(id_range)
		selection[ii].save_score(id_range)
	_index += 1
	seconds = time.time() - start_time
	minutes = int(seconds // 60)
	seconds -= minutes * 60
	print(f'{_index} {minutes}:{int(seconds)}')

for player in selection:
	player._draw_counter = 0
	player._win_counter = 0
	player._loose_counter = 0
	player._total_counter = 0
	for copy in player._copies:
		player._draw_counter += copy[0]._draw_counter + copy[1]._draw_counter
		player._win_counter += copy[0]._win_counter + copy[1]._win_counter
		player._loose_counter += copy[0]._loose_counter + copy[1]._loose_counter
		player._total_counter += copy[0]._total_counter + copy[1]._total_counter

selection.sort(key=lambda p: (-p._win_counter, -p._draw_counter))
with open(f'score_board_{id_range}.log', 'w') as file:
	file.writelines(map(lambda p: f'{p} w: {p._win_counter} d: {p._draw_counter} l: {p._loose_counter}\n', selection))

for player in players:
	for copy in player._copies:
		del copy
	del player._copies
	del player
del players

preparing FINAL
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
